In [ ]:
import asyncio
from dataclasses import dataclass
import multiprocessing as mp
import nest_asyncio
from typing import Any, cast, TypeVar
import uuid

nest_asyncio.apply()


T = TypeVar("T")


@dataclass
class Request:
    id: str
    method_name: str
    args: tuple
    kwargs: dict


@dataclass
class Response:
    id: str
    result: Any
    exception: Exception | None


def move(obj: T) -> T:
    requests = mp.Queue()
    responses = mp.Queue()

    async def handle_request(obj: T, request: Request, responses: mp.Queue) -> Response:
        try:
            result_or_callable = getattr(obj, request.method_name)
            if callable(result_or_callable):
                result_or_coro = result_or_callable(*request.args, **request.kwargs)
                if asyncio.iscoroutine(result_or_coro):
                    result = await result_or_coro
                else:
                    result = result_or_coro
            else:
                result = result_or_callable
            response = Response(request.id, result, None)
        except Exception as e:
            response = Response(request.id, None, e)
        responses.put_nowait(response)

    async def handle_requests(obj: T, requests: mp.Queue, responses: mp.Queue) -> None:
        while True:
            request: Request = await asyncio.get_event_loop().run_in_executor(
                None, requests.get
            )
            asyncio.create_task(handle_request(obj, request, responses))

    def target(obj: T, requests: mp.Queue, responses: mp.Queue) -> None:
        asyncio.run(handle_requests(obj, requests, responses))

    process = mp.Process(target=target, args=(obj, requests, responses))
    process.start()

    futures: dict[str, asyncio.Future] = {}

    async def handle_responses() -> None:
        while True:
            response: Response = await asyncio.get_event_loop().run_in_executor(
                None, responses.get
            )
            future = futures.pop(response.id)
            if response.exception:
                future.set_exception(response.exception)
            else:
                future.set_result(response.result)

    asyncio.create_task(handle_responses())

    class Proxy:
        def __getattr__(self, name: str) -> Any:
            # For attributes that aren't methods, get them directly
            if not hasattr(obj, name):
                raise AttributeError(f"{type(obj).__name__} has no attribute '{name}'")

            async def get_response(args: tuple, kwargs: dict) -> Any:
                request = Request(str(uuid.uuid4()), name, args, kwargs)
                futures[request.id] = asyncio.Future()
                requests.put_nowait(request)
                return await futures[request.id]

            # Check if it's a method or property
            attr = getattr(obj, name)
            if asyncio.iscoroutinefunction(attr):
                # Return an async wrapper function
                async def async_method_wrapper(*args: Any, **kwargs: Any) -> Any:
                    return await get_response(args, kwargs)

                return async_method_wrapper
            elif callable(attr):
                # Return a regular function wrapper
                def method_wrapper(*args: Any, **kwargs: Any) -> Any:
                    return asyncio.run(get_response(args, kwargs))

                return method_wrapper
            else:
                # For non-callable attributes, get them directly
                return asyncio.run(get_response((), {}))

    return cast(T, Proxy())


class Service:
    async def load_unsloth(self) -> None:
        import unsloth

    async def greet(self, name: str, sleep: float) -> str:
        await asyncio.sleep(sleep)
        return f"Hello, {name}!"


service = Service()
service = move(service)
await service.load_unsloth()
await service.greet("World", 1.0)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-29 15:12:27 __init__.py:207] Automatically detected platform cuda.


'Hello, World!'